<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Tic-Tac-Toe
</p><br>
<strong>3x3 version of the <a href='https://en.wikipedia.org/wiki/Hex_(board_game)'>Hex game</a> solved using the minimax method</strong>

Minimax works:
- on games for 2 players
- when a player's win is another player's loss: 0-sum games
- when there is a complete information on the possible outcomes
- where the goal of the game is to minimize loss (and we assume that the other player is trying to maximize gain)

<p>Minimax can either be the brute force method of calculating every possible outcome, or improved using alpha-beta pruning (which eliminates sqrt(n) where n is the number of combinations for the brute-force method for the negamax solution, or n^0.75 otherwise)</p>
<p>We calculate the probability to win for the first player only.</p>

[@vbipin]('https://github.com/vbipin')
The minimax should contain the following functions:
* calculate the children of the current state (<strong>child_node</strong>): or for more complex games with some heuristics, divide into action(state) and move(state,action)
* evaluate if the current state is terminal: either when reaching maximum depth or in case of win/loss (<strong>calc_value()</strong>)
* utility function (<strong>calc_score()</strong>) that interprets the terminal state in terms of value for the player (typically, 1, -1 or 0). To assign different values to paths based on how fast they reach a terminal state, associate to each node a weight corresponding to the distance to the terminal state (a win becomes +1+depth_number, a losss -1-depth_number)
* the minimax is a recursive function calculating the utility function for all the children of a given state, for each state. Since players alternate to play each state, the utility for a given state is reversed from the point of view of the original player.

### Calculate the win/loss function
<a href='http://ohboyigettodomath.blogspot.com/2015/05/tic-tac-toe-as-magic-square.html'>Magic square trick</a>

In [1]:
import numpy as np

In [2]:
class State():
    
    def __init__(self,state):
        self.grid = state
        self.depth = np.sum(np.isin(self.grid,'.'))
        self.itemindex = np.where(np.asanyarray(self.grid) == '.')[0]
        self.id = ''.join([str(i) for i in self.grid])
        self.player = 'X'
        self.next_player = 'O'
        if self.depth % 2 == 0:
            self.player = 'O'
            self.next_player = 'X'
            
    def calc_value(self): 
        numbered_grid = [2, 7, 6, 9, 5, 1, 4, 3, 8]
        self.nb_player = [numbered_grid[k] if x == self.player else 0 for k,x in enumerate(self.grid)]
        self.score_cols = max([sum(self.nb_player[x:x+3]) for x in [0,3,6]])
        self.score_rows=max([sum(self.nb_player[x::3]) for x in range(3)])
        self.score_diagonal1 = self.nb_player[0] + self.nb_player[4] + self.nb_player[8]
        self.score_diagonal2 = self.nb_player[2] + self.nb_player[4] + self.nb_player[6]
        return max(self.score_cols,self.score_rows,self.score_diagonal1,self.score_diagonal2)
    
    def calc_score(self):
        #returns 1 for the maxplayer, -1 for the minplayer, 0 if nobody wins at this state
        if self.calc_value() == 15:
            if self.player == 'O':
                return self.depth + 1
            else:
                return -self.depth - 1
        return 0
    
    def child_node(self):
        #contains a list of all the possible grids at the next move for a player
        child_node = []
        child_grid = self.grid.copy()   
        #replace each remaining 0 by 'player' one by one and append the resulting grid to the child_node list
        for child in self.itemindex:
            child_grid[child] = self.next_player
            child_node.append(State(child_grid))
            child_grid = self.grid.copy()
        return child_node
    
    def terminal(self):
        #evaluate if terminal state (only possible once the first player has played at least 3 times) or a player wins
        return self.depth<6 and self.calc_score()!=0

### Minimax

In [3]:
def minimax(state, scores):
    if state.depth == 0 or state.terminal():
        value=state.calc_score()
    else:
        scores[state.id] = [minimax(child, scores) for child in state.child_node()]
        value = min(scores[state.id])
        if state.player == 'X':
            value = max(scores[state.id])
    return value

In [14]:
def main(state):
    print(np.array(state.grid).reshape(-1, 3))
    scores = {}
    chance = minimax(state, scores)
    print('Best score (if both players maximize their chance): ', chance)
    print('Next move:')
    for branch in scores.values():
        if len(branch)==state.depth:
            for k,v in enumerate(branch):
                ideal = max(branch)
                if state.player=='O':
                    ideal = min(branch)
                if v==ideal:
                    next_move = state.grid.copy()
                    next_move[state.itemindex[k]] = state.next_player
                    print(np.array(next_move).reshape(-1, 3))                    

    results = [item for k,v in scores.items() for item in v]
    print('Moves resulting in a draw: ',"{:.1%}".format(results.count(0) / len(results)))
    print('Moves resulting in a loss: ',"{:.1%}".format(sum(i < 0 for i in results) / len(results)))
    print('Moves resulting in a victory: ',"{:.1%}".format(sum(i > 0 for i in results) / len(results)))
    
    return scores

In [15]:
main(State(['O','.','X','O','X','.', '.','.','.']))

[['O' '.' 'X']
 ['O' 'X' '.']
 ['.' '.' '.']]
Best score (if both players maximize their chance):  5
Next move:
[['O' '.' 'X']
 ['O' 'X' '.']
 ['O' '.' '.']]
Moves resulting in a draw:  0.0%
Moves resulting in a loss:  56.5%
Moves resulting in a victory:  43.5%


{'OOXOXX.O.': [-2, -2],
 'OOXOXX.XO': [1],
 'OOXOXX..O': [-2, 1],
 'OOXOXX...': [3, -2, -2],
 'OOXOXO.XX': [1],
 'OOXOXO.X.': [-2, 1],
 'OOXOX..XO': [1, -2],
 'OOXOX..X.': [-2, 3, -2],
 'OOXOXO..X': [-2, 1],
 'OOXOX..OX': [-2, -2],
 'OOXOX...X': [-2, 3, -2],
 'OOXOX....': [3, -4, 3, 3],
 'OXXOXO.OX': [1],
 'OXXOXO.O.': [-2, 1],
 'OXXOXO..O': [-2, -2],
 'OXXOXO...': [3, -2, -2],
 'O.XOXO.XO': [-2, -2],
 'O.XOXO.X.': [-2, 3, -2],
 'O.XOXO.OX': [1, -2],
 'O.XOXO..X': [-2, 3, -2],
 'O.XOXO...': [3, -4, 3, 3],
 'OXXOXX.OO': [1],
 'OXXOX..OO': [1, -2],
 'OXXOX..O.': [-2, 3, -2],
 'O.XOXX.OO': [1, -2],
 'O.XOXX.O.': [-2, 3, -2],
 'O.XOX..OX': [-2, -2, 3],
 'O.XOX..O.': [3, 3, -4, 3],
 'OXXOX...O': [-2, 3, -2],
 'O.XOXX..O': [-2, 3, -2],
 'O.XOX..XO': [-2, -2, 3],
 'O.XOX...O': [3, 3, -4, 3],
 'O.XOX....': [-4, -4, 5, -4, -4]}

In [95]:
centerstart = main(State(list('....O....')))

[['.' '.' '.']
 ['.' 'O' '.']
 ['.' '.' '.']]
Best score (if both players maximize their chance):  0
Next move:
[['X' '.' '.']
 ['.' 'O' '.']
 ['.' '.' '.']]
[['.' '.' 'X']
 ['.' 'O' '.']
 ['.' '.' '.']]
[['.' '.' '.']
 ['.' 'O' '.']
 ['X' '.' '.']]
[['.' '.' '.']
 ['.' 'O' '.']
 ['.' '.' 'X']]
Moves resulting in a draw:  17.6%
Moves resulting in a loss:  19.8%
Moves resulting in a victory:  62.7%


In [96]:
cornerstart = main(State(list('O........')))

[['O' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
Best score (if both players maximize their chance):  0
Next move:
[['O' '.' '.']
 ['.' 'X' '.']
 ['.' '.' '.']]
Moves resulting in a draw:  18.4%
Moves resulting in a loss:  27.0%
Moves resulting in a victory:  54.6%


In [97]:
middlestart = main(State(list('.O.......')))

[['.' 'O' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
Best score (if both players maximize their chance):  0
Next move:
[['X' 'O' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
[['.' 'O' 'X']
 ['.' '.' '.']
 ['.' '.' '.']]
[['.' 'O' '.']
 ['.' 'X' '.']
 ['.' '.' '.']]
[['.' 'O' '.']
 ['.' '.' '.']
 ['.' 'X' '.']]
Moves resulting in a draw:  19.6%
Moves resulting in a loss:  34.2%
Moves resulting in a victory:  46.2%


### Alpha-beta pruning

In [ ]:
function alphabeta(node, depth, α, β, maximizingPlayer) is
    if depth = 0 or node is a terminal node then
        return the heuristic value of node
    if maximizingPlayer then
        value := −∞
        for each child of node do
            value := max(value, alphabeta(child, depth − 1, α, β, FALSE))
            α := max(α, value)
            if α ≥ β then
                break (* β cut-off *)
        return value
    else
        value := +∞
        for each child of node do
            value := min(value, alphabeta(child, depth − 1, α, β, TRUE))
            β := min(β, value)
            if α ≥ β then
                break (* α cut-off *)
        return value

In [5]:
def alphabeta(state, alpha, beta, scores):
    if state.depth == 0 or state.terminal():
        value=state.calc_score()
        print(state.depth, value)
        return value
    value = float("-inf")
    if state.player == 'X':
        value = float("inf")
    list_scores = []
    for child in state.child_node():
        ab = alphabeta(child, alpha, beta, scores)
        list_scores.append(ab)
        value = max(value, ab)
        alpha = max(alpha, value)
        if state.player == 'X':
            value = min(value, ab)
            beta = min(beta, value)
        print(state.depth, alpha, beta, value)
        if alpha >= beta:
            break
        scores[state.id] = list_scores
    return value

In [9]:
def alphabeta(state, alpha, beta, scores):
    if state.depth == 0 or state.terminal():
        value=state.calc_score()
        print(state.depth, value)
        return value
    if state.player == 'O':
        value = float("-inf")
        list_scores = []
        for child in state.child_node():
            ab = alphabeta(child, alpha, beta, scores)
            list_scores.append(ab)
            value = max(value, ab)
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        scores[state.id] = list_scores
        print(state.depth, alpha, beta, value)
        return value
    elif state.player == 'X':
        value = float("inf")
        list_scores = []
        for child in state.child_node():
            ab = alphabeta(child, alpha, beta, scores)
            list_scores.append(ab)
            value = min(value, ab)
            beta = min(beta, value)
            if alpha >= beta:
                break
        scores[state.id] = list_scores
        print(state.depth, alpha, beta, value)
        return value

In [10]:
def main_alphabeta(state):
    print(np.array(state.grid).reshape(-1, 3))
    scores = {}
    chance = alphabeta(state, float("-inf"), float("inf"), scores)
    print('Best score (if both players maximize their chance): ', chance)
    print('Next move:')
    for branch in scores.values():
        if len(branch)==state.depth:
            for k,v in enumerate(branch):
                ideal = max(branch)
                if state.player=='O':
                    ideal = min(branch)
                if v==ideal:
                    next_move = state.grid.copy()
                    next_move[state.itemindex[k]] = state.next_player
                    print(np.array(next_move).reshape(-1, 3))                    

    results = [item for k,v in scores.items() for item in v]
    print('Moves resulting in a draw: ',"{:.1%}".format(results.count(0) / len(results)))
    print('Moves resulting in a loss: ',"{:.1%}".format(sum(i < 0 for i in results) / len(results)))
    print('Moves resulting in a victory: ',"{:.1%}".format(sum(i > 0 for i in results) / len(results)))
    
    return scores

In [11]:
example = State(['O','.','X','O','X','.', '.','.','.'])
middlestart = State(list('.O.......'))

In [12]:
main_alphabeta(example)

[['O' '.' 'X']
 ['O' 'X' '.']
 ['.' '.' '.']]
2 3
1 -2
1 -2
2 -2 3 -2
1 -2
2 -2 -2 -2
3 -inf -2 -2
3 -4
1 -2
0 1
1 -2 1 1
2 1 inf 1
2 3
0 1
1 -2 1 1
2 1 1 1
3 -2 1 1
1 -2
0 1
1 1 1 1
2 1 inf 1
3 1 1 1
4 1 inf 1
2 3
1 -2
0 1
1 -2 1 1
2 1 1 1
1 -2
1 -2
2 -2 1 -2
3 -inf -2 -2
3 -4
1 -2
0 1
1 -2 1 1
2 1 1 1
2 3
1 -2
1 -2
2 -2 1 -2
3 -2 -2 -2
1 -2
0 1
1 -2 1 1
2 1 1 1
2 3
0 1
1 -2 1 1
2 1 1 1
3 -2 1 1
4 1 1 1
4 5
1 -2
0 1
1 -2 1 1
2 1 1 1
2 3
0 1
1 -inf 1 1
2 1 1 1
3 -inf 1 1
4 1 1 1
1 -2
1 -2
2 -2 1 -2
2 3
0 1
1 -inf -2 1
2 1 -2 1
3 -inf -2 -2
1 -2
0 1
1 -2 1 1
2 1 1 1
2 3
0 1
1 -2 1 1
2 1 1 1
3 -2 1 1
4 1 1 1
5 -inf 1 1
Best score (if both players maximize their chance):  1
Next move:
[['O' '.' 'X']
 ['O' 'X' '.']
 ['O' '.' '.']]
Moves resulting in a draw:  0.0%
Moves resulting in a loss:  33.3%
Moves resulting in a victory:  66.7%


{'OOXOXX.O.': [-2, -2],
 'OOXOXX..O': [-2, 1],
 'OOXOXX...': [3, -2, -2],
 'OOXOXO.XX': [1],
 'OOXOXO.X.': [-2, 1],
 'OOXOXX.XO': [1],
 'OOXOX..XO': [1],
 'OOXOX..X.': [1, 3, 1],
 'OOXOXO..X': [-2, 1],
 'OOXOX...X': [1],
 'OOXOX....': [-2, -4, 1, 1],
 'OXXOXO.OX': [1],
 'OXXOXO.O.': [-2, 1],
 'OXXOXO..O': [-2, -2],
 'OXXOXO...': [3, 1, -2],
 'O.XOXO.XO': [-2, -2],
 'O.XOXO.X.': [1, 3, -2],
 'O.XOXO.OX': [1],
 'O.XOXO..X': [1, 3, 1],
 'O.XOXO...': [-2, -4, -2, 1],
 'OXXOXX.OO': [1],
 'OXXOX..OO': [1],
 'OXXOX..O.': [1, 3, 1],
 'O.XOX..O.': [1],
 'OXXOX...O': [-2, 3, 1],
 'O.XOXX.OO': [1],
 'O.XOXX..O': [1, 3, 1],
 'O.XOX...O': [-2, 1],
 'O.XOX....': [1, 1, 5, 1, 1]}